In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import time
import re
# Multithreading 적용
from concurrent import futures

In [2]:
#DataFrame에 들어갈 list 생성
date_list = []
category_list = []
press_list = []
title_list = []
document_list = []
link_list = []

In [3]:
# 시작, 종료 날짜 현재 기준으로 설정, 페이지값 초기화(이건 나중에 옮기기)
curr_date = datetime.now().strftime("%Y%m%d")
end_date = (datetime.now() - relativedelta(months=1)).strftime("%Y%m%d")

page = 1
count = 1

In [4]:
category_dic = {'정치 일반':'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=269&sid1=100&date=', 
                '경제 일반':'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=263&sid1=101&date=', 
                '사회 일반':'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=257&sid1=102&date=', 
                '생활문화 일반':'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=245&sid1=103&date=', 
                'IT 일반':'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=230&sid1=105&date='}

In [5]:
def call_source(category, search_date, search_page):
    try:
        url = category_dic[category] + search_date + "&page=" + str(search_page)
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        web = requests.get(url, headers=headers).content
        source = BeautifulSoup(web, 'html.parser')

        return source
    except:
        pass

In [6]:
def scrap_news(url, date_list, category_list, press_list, title_list, document_list, link_list):
    try:
        content = requests.get(url).content
        source = BeautifulSoup(content, 'html.parser')

        # 기사날짜
        date = source.find('span', {'class': 'media_end_head_info_datestamp_time _ARTICLE_DATE_TIME'}).get_text()

        # 언론사
        company = source.find('em', {'class' : 'media_end_linked_more_point'}).get_text()
        
        # 기사제목
        title = source.find('h2').get_text()

        # 기사본문
        article = source.find('article', {'id': 'dic_area'}).get_text()
        news_content = article.replace("\n", "")
        news_content = news_content.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
        news_content = news_content.replace("동영상 뉴스       ", "")
        news_content = news_content.replace("동영상 뉴스", "")
        news_content = news_content.strip()

        date_list.append(date)
        category_list.append(category)
        press_list.append(company)
        title_list.append(title)
        document_list.append(news_content)
        link_list.append(url)
        
        time.sleep(0.5)
        
    except:
        pass
    

In [7]:
for key in category_dic:
    # 값 초기화
    category = key
    count = 1
    curr_date = datetime.now().strftime("%Y%m%d")
    print(category, "에 대해서 크롤링을 진행합니다.")

    # Main Code
    with futures.ThreadPoolExecutor(max_workers=8) as executor:
        while curr_date != end_date:
            # page 값 초기화
            page = 1

            # max page 3으로 제한. 총 70페이지 내외
            source = call_source(category, curr_date, 3)
            max_page = int(source.find('div', {'class': "paging"}).find('strong').get_text())
            max_page += 1 # range 함수 고려

            for pg in tqdm(range(1, max_page)):
                # 현재 페이지 정보 불러오기
                source = call_source(category, curr_date, page)
                page_list = source.find_all('a', {'class' : 'nclicks(fls.page)'})

                # 각 기사별 링크 추출
                article_list = []
                article_set = source.find('ul', {'class':'type06_headline'})
                article_set = article_set.find_all('dt')

                for info in article_set:
                    url = info.find('a')['href']
                    article_list.append(url)

                article_list = list(dict.fromkeys(article_list))

                # 각 기사별 스크랩 진행
                for url in article_list:
                    scrap_news(url, date_list, category_list, press_list, title_list, document_list, link_list)

                # 다음 페이지
                page += 1

            # 날짜, page 초기화
            curr_date = (datetime.now() - relativedelta(days=count)).strftime("%Y%m%d")
            count += 1

정치 일반 에 대해서 크롤링을 진행합니다.


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:00<00:00, 20.20s/it]


경제 일반 에 대해서 크롤링을 진행합니다.


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:46<00:00, 15.49s/it]


사회 일반 에 대해서 크롤링을 진행합니다.


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:44<00:00, 14.94s/it]


생활문화 일반 에 대해서 크롤링을 진행합니다.


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:43<00:00, 14.67s/it]


IT 일반 에 대해서 크롤링을 진행합니다.


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:48<00:00, 16.07s/it]


In [8]:
article_df = pd.DataFrame({'Date':date_list,
                          'Category':category_list,
                          'Press':press_list,
                          'Title':title_list,
                          'Document':document_list,
                          'Link':link_list})

# article_df.head()

In [10]:
article_df.head()

,Date,Category,Press,Title,Document,Link
0,2023.10.12. 오전 10:11,정치 일반,MBC,"[시선집중] 이주호 “사교육 부담 증가는 학원가 불안 마케팅.. 올 수능, 킬러문항...","<이주호 사회부총리 겸 교육부장관>\r-대입 제도 개편.. 안정과 공정, 혁신에 최...",https://n.news.naver.com/mnews/article/214/000...
1,2023.10.12. 오전 10:11,정치 일반,세계일보,"대통령실, 보선 참패에 “어떤 선거 결과든 엄중히 받아들여야”","국민의힘, 13일 오전 긴급최고위원회의 개최 “당 체질 개선 논의” 김기현 국...",https://n.news.naver.com/mnews/article/022/000...
2,2023.10.12. 오전 10:09,정치 일반,TV조선,"北, 김영호 통일장관에 ""미치광이·매국 역적"" 막말",북한의 대외 선전매체들이 일제히 김영호 통일부 장관에 대해 막말을 동원하며 비난에 ...,https://n.news.naver.com/mnews/article/448/000...
3,2023.10.12. 오전 10:08,정치 일반,연합뉴스,선서하는 윤희근 경찰청장,(서울=연합뉴스) 김도훈 기자 = 12일 오전 서울 경찰청에서 열린 경찰청·도로교통...,https://n.news.naver.com/mnews/article/001/001...
4,2023.10.12. 오전 10:08,정치 일반,뉴시스,"귀어귀촌 인구유입 사업 예산 전액 삭감…""어촌소멸 나 몰라라""","도시민 어촌유치지원사업·귀어학교 개설사업 내년도 예산 전액 삭감소병훈 위원장 ""핵오...",https://n.news.naver.com/mnews/article/003/001...


In [12]:
len(article_df)

4466

In [11]:
article_df.to_excel('result_{}.xlsx'.format(datetime.now().strftime('%y%m%d_%H%M')), index=False)